# simulate peaks

In [9]:
from peaks_v1 import peakslocation
import numpy as np
import sys
sys.path.append('../PYmodule')
from int_plot import int_sliceviewer

In [10]:

sample =      np.load('../data_sample_44752_5C.npy')
empty =       np.load('../data_empty_44445_5C.npy')
mask_sample = np.load('../mask_sample_44752_bool_5C.npy')
mask_empty =  np.load('../mask_empty_44445_bool_5C.npy')
mask=np.logical_and(mask_sample,mask_empty)
# detector 14 is not counted in sample measreument (run # 44752)

sample_trim=sample[1:,1:500,:251]
empty_trim=empty[1:,1:500,:251]
mask_trim=mask[1:,1:500,:251]



In [12]:

from multiprocessing import Process,Pool

simulatedpeaksdata=np.zeros(sample_trim.shape)
xind_offset=250
yind_offset=250
zind_offset=250
xgridlen=12.5/250
ygridlen=12.5/250
zgridlen=12.5/250
gridlen=0.05
gaussian_width=0.07/gridlen
gaussian_height=1000
peakslocation_converted=np.array(peakslocation)/gridlen+np.array([xind_offset,yind_offset,zind_offset])



#sp1d=[]
#for ind,x in np.ndenumerate(simulatedpeaksdata):
#    sp1d.append(ind)


In [15]:

for peakloc in peakslocation_converted:

    peaklocind=[int(i) for i in peakloc]
    for xind in range(-10,11):
        for yind in range(-10,11):
            for zind in range(-10,11):
                rel_ind=(xind,yind,zind)
                r=np.array(rel_ind)+np.array(peaklocind)-peakloc

                rdis=np.linalg.norm(r)
                
                ab_ind=[int(i) for i in np.array(rel_ind)+np.array(peaklocind)]
                if (ab_ind[0]<499 and ab_ind[1]<498 and ab_ind[2]<250):
                    ind=tuple(ab_ind)
                    if (mask_trim[ind]):
                        simulatedpeaksdata[ind]=np.exp(-(rdis/gaussian_width)**2)*gaussian_height

        


In [ ]:
int_sliceviewer(simulatedpeaksdata)
# change axis to slice the 3D volume along different dimensions
# change index number to view different slices, e.g. peaks should be visible ~x=40, 

interactive(children=(Dropdown(description='Axis:', options=('X', 'Y', 'Z'), value='X'), IntSlider(value=0, de…

In [17]:
np.save('../save_nc_simpeak.npy',simulatedpeaksdata)